In [ ]:
# import sys
# !{sys.executable} -m pip install transformers datasets transformers[torch] accelerate sentencepiece

# https://huggingface.co/spaces/evaluate-metric/sacrebleu
# https://huggingface.co/spaces/evaluate-metric/bertscore
# !{sys.executable} -m pip install evaluate sacrebleu bert_score

In [27]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import evaluate

Translate the FR test set to English using the [NLLB-200](https://huggingface.co/facebook/nllb-200-distilled-600M) model:

In [ ]:
! cd ../scripts && \
    python translate_fr2en.py --input-path "../data/mtob_domain_en_fr_test.csv" --output-path "../data/mtob_domain_en2fr_nllb_test.csv" --model-name "nllb"

Load the output generated by the above script:

In [10]:
test_df = pd.read_csv("../data/mtob_domain_en2fr_nllb_test.csv")

In [11]:
test_df.head(1)

,id,text_en,text_fr,label,label_text,text_fr2en
0,3139393736393839,Show me dates for music festivals in 2018,Montre-moi les dates des festivals de musique ...,2,event,Show me the dates of the music festivals in 2018


Evaluate the translation quality using the [BLEU score](https://huggingface.co/spaces/evaluate-metric/sacrebleu):

In [28]:
sacrebleu = evaluate.load("sacrebleu")

translations = test_df["text_fr2en"].tolist()
references = [[t] for t in test_df["text_en"].values]

results = sacrebleu.compute(predictions=translations, references=references, lowercase=True)
print("BLEU score for FR -> EN:", round(results["score"], 1))

BLEU score for FR -> EN: 41.4


Evaluate the translation quality using [BERTScore](https://huggingface.co/spaces/evaluate-metric/bertscore):

In [33]:
bertscore = evaluate.load("bertscore")

translations = test_df["text_fr2en"].tolist()
references = test_df["text_en"].tolist()

results = bertscore.compute(predictions=translations, references=references, model_type="distilbert-base-uncased")
print("BERT score for FR -> EN:", round(np.mean(results["f1"]), 4))

BERT score for FR -> EN: 0.9167
